In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
from time import time
from copy import deepcopy
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ROOT_PATH = Path().absolute().parent.parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
from frequency_response import FrequencyResponse
from peq import PEQ
from constants import DEFAULT_FS, PEQ_CONFIGS

In [4]:
dir_path = Path().absolute()
config_names = [
    ['4_PEAKING_WITH_LOW_SHELF'],
    ['4_PEAKING_WITH_LOW_SHELF', '4_PEAKING_WITH_HIGH_SHELF'],
    ['8_PEAKING_WITH_SHELVES'],
    ['10_BAND_GRAPHIC_EQ'],
]

In [5]:
def rmse(peq):
    ix10k = np.argmin(np.abs(10000 - peq.f))
    return np.sqrt(np.mean(np.square(peq.target[:ix10k] - peq.fr[:ix10k])))

In [7]:
def band_reqularization_stats(configs, max_n=None):
    df = []
    n = 0
    for fp in ROOT_PATH.joinpath('results', 'oratory1990').glob('*/*/*.csv'):
        fr = FrequencyResponse.read_from_csv(fp)
        t = time()
        peqs = fr.optimize_parametric_eq(configs, DEFAULT_FS, max_time=5)
        df.append([time() - t, rmse(peqs[-1])])
        n += 1
        if max_n is not None and n >= max_n:
            break
    df = pd.DataFrame(df, columns=['Time', 'RMSE'])
    return df.describe()

In [23]:
# Comment out band penalties from filters in PEQ._optimizer_loss()
band_reqularization_stats([PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF']], max_n=10)

,Time,RMSE
count,10.000000,10.000000
mean,0.095721,0.454452
std,0.038987,0.177448
min,0.045010,0.235143
25%,0.065765,0.363119
50%,0.085519,0.436621
75%,0.131029,0.486728
max,0.152034,0.895490


In [24]:
# Do NOT comment out band penalties from filters in PEQ._optimizer_loss()
band_reqularization_stats([PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF']], max_n=10)

,Time,RMSE
count,10.000000,10.000000
mean,0.161436,0.602648
std,0.063334,0.198441
min,0.093020,0.287875
25%,0.109525,0.467750
50%,0.145533,0.606893
75%,0.202546,0.673420
max,0.282064,0.932547


In [25]:
 # Comment out band penalties from filters in PEQ._optimizer_loss()
band_reqularization_stats([PEQ_CONFIGS['8_PEAKING_WITH_SHELVES']], max_n=10)

,Time,RMSE
count,10.000000,10.000000
mean,0.252157,0.206468
std,0.054596,0.055044
min,0.191043,0.144431
25%,0.215049,0.171290
50%,0.248556,0.201377
75%,0.262309,0.214941
max,0.350079,0.321155


In [26]:
# Do NOT comment out band penalties from filters in PEQ._optimizer_loss()
band_reqularization_stats([PEQ_CONFIGS['8_PEAKING_WITH_SHELVES']], max_n=10)

,Time,RMSE
count,10.000000,10.000000
mean,0.547223,0.352217
std,0.129069,0.172374
min,0.326073,0.219051
25%,0.474608,0.249850
50%,0.554625,0.283438
75%,0.633393,0.366972
max,0.746168,0.790463
